In [1]:
import importlib
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')

from __init__ import *
from __init__jupyterlab import *
import datetime
from scipy import sparse
import collections
import itertools
import re
import fbpca
import pickle
import scipy.cluster.hierarchy as sch
from sklearn.metrics import euclidean_distances
import h5py

import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_run_tsne
import CEMBA_clst_utils
import SCF_utils
importlib.reload(SCF_utils)

import seaborn as sns 
import matplotlib.pyplot as plt

- Explore cell numbers (data_freeze_neurons)
- Select cells
- SCF
- Post-SCF evaluation

In [8]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/cell_cluster_assignment_all_integrations.tsv'
meta_grand = pd.read_csv(f, sep='\t')
print(meta_grand.shape)
meta_grand.head()

/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(571703, 7)


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0,it_8,DNA methylation
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1,it_0,DNA methylation
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1,it_0,DNA methylation
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1,it_0,DNA methylation
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1,it_0,DNA methylation


In [24]:
cond1 = meta_grand['SCF_round2'].fillna('').str.contains(r'^9-')
meta_grand['isPT_SCF'] = 0
meta_grand.loc[cond1, 'isPT_SCF'] = 1

_x = meta_grand[cond1]
print(_x.shape)
_x.groupby(['modality', 'SCF_round2']).size().unstack().fillna(0).astype(int)

(3135, 9)


SCF_round2,9-1,9-2
modality,,
10X_cells_v2_AIBS,268,244
10X_cells_v3_AIBS,88,58
10X_nuclei_v3_AIBS,223,240
10X_nuclei_v3_Broad,750,509
DNA methylation,45,52
Open chromatin,563,8
SmartSeq_cells_AIBS,8,4
SmartSeq_nuclei_AIBS,45,30


In [25]:
cond1 = meta_grand['RNA concensus'].fillna(-1).astype(int).isin([70, 71, 72, 73])
meta_grand['isPT_RNAconcensus'] = 0
meta_grand.loc[cond1, 'isPT_RNAconcensus'] = 1

_x = meta_grand[cond1]
print(_x.shape)
_x.groupby(['modality', 'RNA concensus']).size().unstack().fillna(0).astype(int)

(9625, 9)


RNA concensus,70.0,71.0,72.0,73.0
modality,,,,
10X_cells_v2_AIBS,263,129,19,113
10X_cells_v3_AIBS,88,47,7,9
10X_nuclei_v2_AIBS,1688,384,270,634
10X_nuclei_v3_AIBS,716,216,88,227
10X_nuclei_v3_Broad,2943,414,343,820
SmartSeq_cells_AIBS,8,1,0,3
SmartSeq_nuclei_AIBS,93,20,33,49


In [26]:
cond1 = meta_grand['LIGER_level2'].fillna('').isin(['non_it_4', 'non_it_9'])
meta_grand['isPT_LIGER'] = 0
meta_grand.loc[cond1, 'isPT_LIGER'] = 1

_x = meta_grand[cond1]
print(_x.shape)
_x.groupby(['modality', 'LIGER_level2']).size().unstack().fillna(0).astype(int)

(8481, 10)


LIGER_level2,non_it_4,non_it_9
modality,,
10X_cells_v2_AIBS,407,104
10X_cells_v3_AIBS,104,45
10X_nuclei_v3_AIBS,1168,0
10X_nuclei_v3_Broad,3227,1219
DNA methylation,223,54
Open chromatin,1725,0
SmartSeq_cells_AIBS,9,4
SmartSeq_nuclei_AIBS,190,2


In [28]:
meta_ispt = meta_grand[['cell_id', 'modality', 
                        'isPT_SCF', 'isPT_RNAconcensus', 'isPT_LIGER',
                       ]]
print(meta_ispt.shape)
meta_ispt.head()

(571703, 5)


,cell_id,modality,isPT_SCF,isPT_RNAconcensus,isPT_LIGER
0,snmcseq_gene_2C_M_0,DNA methylation,0,0,0
1,snmcseq_gene_2C_M_1,DNA methylation,0,0,0
2,snmcseq_gene_2C_M_100,DNA methylation,0,0,0
3,snmcseq_gene_2C_M_1000,DNA methylation,0,0,0
4,snmcseq_gene_2C_M_1001,DNA methylation,0,0,0


In [39]:
results = meta_ispt.groupby(['modality', 
                               'isPT_SCF', 
                               'isPT_RNAconcensus', 
                               'isPT_LIGER',
                              ]).size().reset_index().rename(columns={0: 'num'})
# results = results[~((results['isPT_SCF']==0) 
#                  & (results['isPT_LIGER']==0) 
#                  & (results['isPT_RNAconcensus']==0)
#                 )]
results['cond_code'] = (results['isPT_SCF'].astype(str)
                        + results['isPT_RNAconcensus'].astype(str)
                        + results['isPT_LIGER'].astype(str)
                       )
print(results.shape)
results

(44, 6)


,modality,isPT_SCF,isPT_RNAconcensus,isPT_LIGER,num,cond_code
0,10X_cells_v2_AIBS,0,0,0,122116,000
1,10X_cells_v2_AIBS,0,0,1,1,001
2,10X_cells_v2_AIBS,0,1,0,5,010
3,10X_cells_v2_AIBS,0,1,1,7,011
4,10X_cells_v2_AIBS,1,1,0,9,110
5,10X_cells_v2_AIBS,1,1,1,503,111
6,10X_cells_v3_AIBS,0,0,0,71025,000
7,10X_cells_v3_AIBS,0,0,1,5,001
8,10X_cells_v3_AIBS,0,1,0,3,010
9,10X_cells_v3_AIBS,0,1,1,4,011


In [51]:
decode_table = {
    '000': "None",
    '001': "LIGER only",
    '010': "RNA concensus only",
    '100': "SCF only",
    '110': "SCF and RNA",
    '101': "SCF and LIGER",
    '011': "RNA and LIGER",
    '111': "all",
}

datasets = [
    'SmartSeq_cells_AIBS', 
    '10X_cells_v3_AIBS', 
    '10X_cells_v2_AIBS', 
    'SmartSeq_nuclei_AIBS', 
    '10X_nuclei_v3_AIBS', 
    '10X_nuclei_v3_Broad', 
    '10X_nuclei_v2_AIBS', 
    'DNA methylation', 
    'Open chromatin', 
]

results_summary = (results.pivot(columns='cond_code', 
                                 index='modality', 
                                 values='num')
                            .fillna(0).astype(int)
                    )[['000', '100', '010', '001', 
                       '110', '101', '011', '111']].reindex(datasets).rename(columns=decode_table)
results_summary.columns.name = 'isPT?'
print(results_summary.shape)
results_summary

(9, 8)


isPT?,None,SCF only,RNA concensus only,LIGER only,SCF and RNA,SCF and LIGER,RNA and LIGER,all
modality,,,,,,,,
SmartSeq_cells_AIBS,6275,0,0,1,0,0,0,12
10X_cells_v3_AIBS,71025,2,3,5,4,0,4,140
10X_cells_v2_AIBS,122116,0,5,1,9,0,7,503
SmartSeq_nuclei_AIBS,5976,0,2,0,1,0,118,74
10X_nuclei_v3_AIBS,38918,1,43,0,36,0,742,426
10X_nuclei_v3_Broad,155216,1,56,1,19,0,3206,1239
10X_nuclei_v2_AIBS,73549,0,2976,0,0,0,0,0
DNA methylation,9068,21,0,201,0,76,0,0
Open chromatin,77828,72,0,1226,0,499,0,0


# Track information

In [78]:
f = '/cndd2/fangming/projects/miniatlas/results/l5pt_candidates.tsv'

In [79]:
meta_pt = meta_grand[meta_grand.filter(regex=r'^isPT', axis=1).sum(axis=1) > 0]
print(meta_pt.shape)

meta_pt = meta_pt[~meta_pt['SCF_round2'].isnull()]
print(meta_pt.shape)
meta_pt.head()

(11732, 10)
(8756, 10)


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality,isPT_SCF,isPT_RNAconcensus,isPT_LIGER
57,snmcseq_gene_2C_M_1054,9-2,9-2-2,NaN,12,non_it_4,DNA methylation,1,0,1
63,snmcseq_gene_2C_M_106,3-2,3-2-1,NaN,12,non_it_4,DNA methylation,0,0,1
81,snmcseq_gene_2C_M_1076,3-2,3-2-1,NaN,12,non_it_4,DNA methylation,0,0,1
83,snmcseq_gene_2C_M_1078,9-2,9-2-1,NaN,12,non_it_4,DNA methylation,1,0,1
114,snmcseq_gene_2C_M_1110,9-2,9-2-1,NaN,12,non_it_4,DNA methylation,1,0,1


In [80]:
meta_pt['modality'].value_counts()

10X_nuclei_v3_Broad     4522
Open chromatin          1797
10X_nuclei_v3_AIBS      1248
10X_cells_v2_AIBS        525
DNA methylation          298
SmartSeq_nuclei_AIBS     195
10X_cells_v3_AIBS        158
SmartSeq_cells_AIBS       13
Name: modality, dtype: int64

In [81]:
meta_pt.to_csv(f, sep='\t', header=True, index=False)

# Second section

In [3]:
data_dir = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/'

### Number of mC PT cells? 

In [9]:
mod = 'snmcseq_gene'
f = os.path.join(data_dir, '{}_metadata.tsv'.format(mod))
meta = pd.read_csv(f, sep='\t')
print(meta.shape)
meta.head()

(9876, 32)


,cell,FullCellID,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,...,Slice,PassFilter,pca_0,pca_1,umap_0,umap_1,tsne_0,tsne_1,MajorCluster,SubCluster
0,2C_M_0,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00511,0.77073,0.76955,0.02154,0.01651,1076259.0,2314862,1517338.0,...,2,True,-11.588627,-0.005045,10.051389,7.099836,-3.343437,18.330174,L4-IT-Rorb,L4-IT-Rorb_Rorb-Tenm2
1,2C_M_1,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00531,0.75683,0.75553,0.02196,0.01674,1197092.0,2590626,1709334.0,...,2,True,-19.267288,-7.374098,13.717241,-1.786627,9.212860,-26.514785,L23-IT-Cux2,L23-IT-Cux2
2,2C_M_10,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00581,0.75720,0.75578,0.02603,0.02034,1006630.0,2348484,1606410.0,...,2,True,-15.340144,-2.739380,4.829485,9.943986,-22.960995,13.509483,Outlier,Outlier
3,2C_M_100,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00634,0.74984,0.74824,0.02718,0.02097,2026078.0,4599922,3154619.0,...,2,True,-23.625140,-5.929033,13.739521,1.107174,12.798545,-17.126331,L23-IT-Cux2,L23-IT-Cux2
4,2C_M_1000,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00521,0.77073,0.76953,0.02117,0.01604,2368464.0,5320502,3648786.0,...,2,True,-21.815190,-7.130878,14.154541,-2.284118,8.443658,-27.943267,L23-IT-Cux2,L23-IT-Cux2


In [54]:
print(meta['MajorCluster'].str.contains(r'^L5-PT').sum())

354


### Number of ATAC PT cells? 

In [55]:
mod = 'snatac_gene'
f = os.path.join(data_dir, '{}_metadata.tsv'.format(mod))
meta = pd.read_csv(f, sep='\t')
print(meta.shape)
meta.head()

(81196, 13)


,cell,sample,barcode,TN,UM,PP,UQ,CM,cluster,tsne1,tsne2,umap-1,umap-2
0,CEMBA171206_3C_AGCGATAGAACCAGGTAAGAGATGTATAGCCT,CEMBA171206_3C,AGCGATAGAACCAGGTAAGAGATGTATAGCCT,138330,128263,128039,64868,0,L6.CT,7.605117,0.697672,-2.125204,-4.396653
1,CEMBA171206_3C_AGCGATAGAACCAGGTAATGACGTCAGGACGT,CEMBA171206_3C,AGCGATAGAACCAGGTAATGACGTCAGGACGT,159133,146663,146348,69167,0,L23.b,-9.492925,-6.200938,8.959074,3.006819
2,CEMBA171206_3C_AGCGATAGAACCAGGTAGGATAACATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTAGGATAACATAGAGGC,19152,17586,17545,8348,0,ASC,14.227358,-11.023748,-6.880754,-2.903271
3,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,14876,13837,13801,6583,0,L6.CT,5.144768,4.790674,-0.955707,-4.635163
4,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTATAGAGGC,28532,26303,26202,12437,0,L5.IT.b,5.218895,-8.674734,0.796851,4.447933


In [56]:
meta['cluster'].unique()

array(['L6.CT', 'L23.b', 'ASC', 'L5.IT.b', 'Other', 'L4', 'Pv_Ntf3_Trim63',
       'NP', 'MGC', 'L5.IT.a', 'L23.a', 'OPC', 'OGC', 'L5.PT', 'Sst_Man1a',
       'L6.IT', 'Lamp5_Arhgdib', 'L23.c', 'Pv_Tac1', 'Lamp5_Smad3', 'Endo',
       'Vip_Hcls1', 'Sst_Chrna2_Myh8', 'Sst_Stk33', 'Vip_Chat', 'Pv_Vsig2',
       'Sncg', 'Vip_Gcnt4', 'Smc', 'Chodl', 'Lamp5_Ndnf', 'Vip_Lipg',
       'Lamp5_Mettl21e'], dtype=object)

In [57]:
meta['cluster'].str.contains(r'^L5.PT').sum()

1755